In [2]:
# DSPy demo goes here
# see https://github.com/ericmelz/dspy/blob/main/intro.ipynb

In [9]:
import dspy
import openai

In [10]:

f = open('/Users/ericmelz/Documents/code/openai-quickstart-python-emelz/.env', 'r')

KEY='OPENAI_API_KEY'
l=f.readline()
api_key = 'undefined!'
while l:
    if l.startswith(KEY):
        print('FOUND')
        api_key = l[len(KEY)+1:-1]
        break
    l = f.readline()

f.close()

openai.api_key=api_key

FOUND


In [4]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)

In [5]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [6]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

In [13]:
# Call the predictor on a particular input.
pred = generate_answer(question='What is the capital of France?')

In [14]:
pred

Prediction(
    answer='Paris'
)

## Some math using BasicQA and turbo

In [38]:
def expect(gold, pred):
    if str(gold) == str(pred.answer):
        print('correct')
    else:
        print(F'INCORRECT.  Expected {gold}, got {pred.answer}')

In [27]:
pred = generate_answer(question='2+2')
expect(4, pred)

correct


In [28]:
pred = generate_answer(question='7+9')
expect(16, pred)

correct


In [31]:
pred = generate_answer(question='27+59')
expect(86, pred)

correct


In [33]:
pred = generate_answer(question='327+959')
expect(1286, pred)

correct


In [34]:
pred = generate_answer(question='4327+8959')
expect(13286, pred)

correct


In [35]:
pred = generate_answer(question='4329+8959')
expect(13288, pred)

correct


In [36]:
pred = generate_answer(question='44329+28959')
expect(73288, pred)

correct


In [39]:
pred = generate_answer(question='944329+928959')
expect(1873288, pred)

INCORRECT.  Expected 1873288, got 1876288


## CommonsenseQA
See https://github.com/ezelikman/STaR/blob/main/commonsenseqa/prompts.txt

In [44]:
q = """
Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
"""

pred = generate_answer(question=q)
expect('(e) blotter', pred)

correct


In [46]:
q = """
Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk
"""
a = "(c) television"

expect(a, generate_answer(question=q))

INCORRECT.  Expected (c) television, got television


In [53]:
preamble = """
Answer the following multiple choice question.

Present the answer in the format
(${letter}) ${answer}

----
Example:

Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter

A: (e) blotter
----
"""

In [54]:
q = f"""
{preamble}
Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk

A:
"""
a = "(c) television"

expect(a, generate_answer(question=q))

correct


In [55]:
q = f"""
{preamble}
Q: The fox walked from the city into the forest, what was it looking for?
Answer Choices:
(a) pretty flowers
(b) hen house
(c) natural habitat
(d) storybook
(e) dense forest

A:
"""
a = "(c) natural habitat"

expect(a, generate_answer(question=q))

correct


In [56]:
q = f"""
{preamble}
Q: Sammy wanted to go to where the people were. Where might he go?
Answer Choices:
(a) populated areas
(b) race track
(c) desert
(d) apartment
(e) roadblock

A:
"""
a = "(a) populated areas"

expect(a, generate_answer(question=q))

correct


In [57]:
q = f"""
{preamble}
Q: Where do you put your grapes just before checking out?
Answer Choices:
(a) mouth
(b) grocery cart
(c) super market
(d) fruit basket
(e) fruit market

A:
"""
a = "(b) grocery cart"

expect(a, generate_answer(question=q))

correct


In [59]:
q = f"""
{preamble}
Q: Google Maps and other highway and street GPS services have replaced what?
Answer Choices:
(a) united states
(b) mexico
(c) countryside
(d) atlas
(e) oceans

A:
"""
a = "(d) atlas"

expect(a, generate_answer(question=q))

correct


In [60]:
q = f"""
{preamble}
Q: Before getting a divorce, what did the wife feel who was doing all the work?
Answer Choices:
(a) harder
(b) anguish
(c) bitterness
(d) tears
(e) sadness

A:
"""
a = "(c) bitterness"

expect(a, generate_answer(question=q))

INCORRECT.  Expected (c) bitterness, got (a) harder
